# Keras classification

Example how to use Keras for solving classification task.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

## Downloading the data

We will use fashion MNIST dataset - 10 types of clothes on small pictures.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Training set.

In [ ]:
train_images.shape

Test set.

In [ ]:
test_images.shape

Train labels.

In [ ]:
train_labels

Defining class names.

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Plotting data sample.

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    # setting subplot - each image is separate subplot in 5x5 matrix
    plt.subplot(5, 5, i+1)
    # do not show ticks on axes
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

Plotting the image.

In [ ]:
plt.figure()
# plot image in grayscale
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.show()

## Normalizing data

Images are on grayscale ranging from 0 to 255, so we will normalize the data by dividing by 255 to get range from 0-1.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.show()

## Setting up model architecture

We are setting up deep fully connected neural network using functional API.

In [ ]:
# we are using 2D data, image of 28x28 pixels, so we need to reflect that in the input shape
input_layer = Input(shape=(28, 28))
# we do not use 2D convolutions, so we are flattening the data to 1D vector with 784 components
x = Flatten()(input_layer)
# 128 units of fully connected network with ReLu activation function
x = Dense(128, activation='relu')(x)
# we have 10 classes, so we will set Dense layer as an output layer with 10 units and softmax activation function
output_layer = Dense(10, activation='softmax')(x)
model = Model(inputs = input_layer, outputs = output_layer)
model.summary()

## Training of the model

We are using adam optimizer again and SparseCategoricalCrossentropy - used when class labels are integers. For one hot encoding of the labels we would use CategoricalCrossentropy.

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

Training is running for 10 epochs.

In [ ]:
%%time
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

## Training results

In [ ]:
# loss function on training set
plt.plot(history.history['loss'], label='loss')
# loss function on validation set
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss function')
plt.grid(True)

In [ ]:
# accurancy on training set
plt.plot(history.history['accuracy'], label='accuracy')
# accurancy on validation set
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.title('Accuracy')
plt.grid(True)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f'Test accuracy: {100*test_acc:.2f}%')

## Working with single prediction

Since we are predicting on batches, we need to add one dimension in case of predicting single picture.

In [ ]:
prediction = model.predict(np.expand_dims(test_images[0], 0))

How prediction looks like?

In [ ]:
prediction

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.title("{} {:2.0f}% ({})".format(class_names[np.argmax(prediction)],
                                         100*np.max(prediction),
                                         class_names[test_labels[0]]), color='blue')
plt.show()

## Running predictions on the whole test set

In [ ]:
%%time
predictions = model.predict(test_images)

Helper function for plotting images with predicted and true labels.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    # ground truth label
    true_label = true_label[i]
    # image used for prediction
    img = img[i]
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)
    # taking index of maximum prediction value
    predicted_label = np.argmax(predictions_array)
    # if predicted label is the same as a ground truth, use blue color, red otherwise
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'
    # just formating label
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(predictions_array),
                                         class_names[true_label]),color=color)

Bar plots with predictions.

In [ ]:
def plot_value_array(i, predictions_array, true_label):
    # ground truth
    true_label = true_label[i]
    # we have 10 classes, so tick for every class
    plt.xticks(range(10))
    plt.yticks([])
    # plot all predictions using bar plots
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    # softmax sum = 1, so there is no point to have bigger scale
    plt.ylim([0, 1])
    # prediction with maximum confidence
    predicted_label = np.argmax(predictions_array)
    # set colors for bad prediction
    thisplot[predicted_label].set_color('red')
    # set color for good prediction
    thisplot[true_label].set_color('blue')

## Plotting results

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()